In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#파일 경로 보기
os.listdir('../input/aerial-cactus-identification')

In [ ]:
os.listdir()

In [ ]:
import zipfile 

test_zip = zipfile.ZipFile('../input/aerial-cactus-identification/test.zip')
test = test_zip.extractall('')

test_zip.close()

train_zip = zipfile.ZipFile('../input/aerial-cactus-identification/train.zip')
train = train_zip.extractall('')

In [ ]:
os.listdir('train')

In [ ]:
train_dir ='train'

In [ ]:
csv_path ='../input/aerial-cactus-identification/train.csv'

df = pd.read_csv(csv_path)

df.head()

In [ ]:
# 텐서플로우에서 하나씩 읽는 모듈을 이용해서 학습시켜보자 

# 먼저 각 아이디를 경로로 바꿔주자
filenames = df['id']
filenames

In [ ]:
file_paths = [os.path.join(train_dir, fname) for fname in filenames]

In [ ]:
file_paths

In [ ]:
train_df = pd.DataFrame(data ={'id':file_paths,
                              'has_cactus':df['has_cactus']})

In [ ]:
train_df.head()

In [ ]:
# 참고로 텐서플로우는 숫자 타입이 아닌 문자 타입으로 받아야 된다 

train_df = train_df.astype(np.str)

In [ ]:
# train set 안에서 test, train을 나눠주려고 한다 
# 기존의 test 셋에는 정답 데이터가 없기 떄문 

len(train_df)

In [ ]:
train_df = train_df[:-500]
test_df = train_df[-500:]

len(train_df), len(test_df)

In [ ]:
path = train_df['id'][0]

# Load Package

In [ ]:
from tqdm  import tqdm_notebook

import matplotlib.pyplot as plt 
from PIL import Image

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_dir

In [ ]:
test_dir = 'test'

# Data Preporcess

In [ ]:
path

In [ ]:
# 이미지 보기 

img_pil = Image.open(path)
image = np.array(img_pil)

image.shape

In [ ]:
plt.imshow(image)

In [ ]:
# 하이퍼 파라미터 설정 

input_shape = (32,32,3)
batch_size = 32
num_classes = 2
num_epochs =10 

learning_rate = 0.01

In [ ]:
# 모델 

inputs = layers.Input(input_shape)
net = layers.Conv2D(64,(3,3), padding ='same')(inputs)
net = layers.Conv2D(64,(3,3), padding ='same')(net)
net = layers.Conv2D(64,(3,3), padding ='same')(net)
net = layers.BatchNormalization()(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2,2))(net)


net = layers.Conv2D(128,(3,3), padding ='same')(inputs)
net = layers.Conv2D(128,(3,3), padding ='same')(net)
net = layers.Conv2D(128,(3,3), padding ='same')(net)
net = layers.BatchNormalization()(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2,2))(net)
net = layers.Dropout(0.25)(net)

net = layers.Conv2D(256,(3,3), padding ='same')(inputs)
net = layers.Conv2D(256,(3,3), padding ='same')(net)
net = layers.Conv2D(256,(3,3), padding ='same')(net)
net = layers.BatchNormalization()(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2,2))(net)
net = layers.Dropout(0.25)(net)

net = layers.Conv2D(512,(3,3), padding ='same')(inputs)
net = layers.Conv2D(512,(3,3), padding ='same')(net)
net = layers.Conv2D(512,(3,3), padding ='same')(net)
net = layers.BatchNormalization()(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2,2))(net)
net = layers.Dropout(0.25)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(0.5)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs= inputs, outputs=net)

In [ ]:
model.compile(loss = 'sparse_categorical_crossentropy',
             optimizer= tf.keras.optimizers.Adam(learning_rate),
             metrics=['accuracy'])

In [ ]:
#Data Preprocess 2 

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255.,
    width_shift_range = 0.3,
    zoom_range=0.2,
    horizontal_flip = True
)

test_datagen = ImageDataGenerator(
    rescale=1./255.

)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col = 'id',
    y_col = 'has_cactus',
    target_size = input_shape[:2],
    batch_size=batch_size,
    class_mode = 'sparse'
    
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col = 'id',
    y_col = 'has_cactus',
    target_size = input_shape[:2],
    batch_size=batch_size,
    class_mode = 'sparse'
    
)

## Train

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch = len(train_generator),
    epochs = num_epochs,
    validation_data = test_generator,
    validation_steps = len(test_generator))